### Import

In [26]:
from dataclasses import dataclass, field, asdict
from typing import List
from csv2shex.csvreader import (
    csvreader, 
    _get_csvrow_dicts_list,
    _get_corrected_csvrows_list, 
    _get_csvshape_dicts_list, 
)
from csv2shex.csvrow import CSVRow
from csv2shex.utils import pprint_df
import pandas as pd

### Declare

In [27]:
@dataclass
class CSVTripleConstraint:
    """Instances hold TAP/CSV row elements that form a triple constraint."""
    propertyID: str = None
    valueConstraint: str = None
    valueShape: str = None
    extras: field(default_factory=dict) = None
    # propertyLabel: str = None
    # mandatory: str = None
    # repeatable: str = None
    # valueNodeType: str = None
    # valueDataType: str = None
    # valueConstraintType: str = None
    # note: str = None

@dataclass
class CSVShape:
    """Instances hold TAP/CSV row elements that form a shape."""
    shapeID: str = None
    # shapeLabel: str = None
    # shapeClosed: str = None
    # start: bool = False
    tripleconstraints_list: List[CSVTripleConstraint] = field(default_factory=list)

@dataclass
class CSVSchema:
    """Set of shapes."""

In [28]:
csvrow_dicts_list = [{'shapeID': ':book',
  'propertyID': 'dc:creator',
  'valueConstraint': '',
  'valueShape': ':author'},
 {'shapeID': '',
  'propertyID': 'dc:type',
  'valueConstraint': 'so:Book',
  'valueShape': ''},
 {'shapeID': ':author',
  'propertyID': 'foaf:name',
  'valueConstraint': '',
  'valueShape': ''}]

### For each row
#### 1. Initialize instance of CSVShape

In [29]:
for row in csvrow_dicts_list:
    shape = CSVShape()
    shape.shapeID = row["shapeID"]
    shape.tripleconstraints_list = list()

dict_of_shape_objs[shape_dict["shapeID"]] = shape
dict_of_shape_objs

{'b': CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[]),
 'c': CSVShape(shapeID=':author', tripleconstraints_list=[])}

#### 2. On finding new shapeID, capture shape-related elements in a shape_dict.

In [4]:
shape_dict = dict()
shape_dict["shapeID"] = "b"
shape_dict["shapeLabel"] = "label"
shape_dict["shapeClosed"] = False
shape_dict["start"] = True
shape_dict["tripleconstraints_list"] = list()
shape_dict

{'shapeID': 'b',
 'shapeLabel': 'label',
 'shapeClosed': False,
 'start': True,
 'tripleconstraints_list': []}

#### 3. Assign CSVShape instance as value to key "shapeID" in dict_of_shape_objs

In [8]:
dict_of_shape_objs = dict()
dict_of_shape_objs[shape_dict["shapeID"]] = cshap
dict_of_shape_objs

{'b': CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[])}

In [9]:
"b" in dict_of_shape_objs

True

In [10]:
# Triple constraints list for shape "b"
dict_of_shape_objs["b"].tripleconstraints_list

[]

#### 4. Each new shape is added to dict_of_shape_dicts.

In [13]:
shape_dict = dict()
shape_dict["shapeID"] = "c"
shape_dict["shapeLabel"] = "clabel"
shape_dict["shapeClosed"] = False
shape_dict["start"] = False
shape_dict["tripleconstraints_list"] = list()

dict_of_shape_objs[shape_dict["shapeID"]] = CSVShape(**shape_dict)
dict_of_shape_objs

{'b': CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[]),
 'c': CSVShape(shapeID='c', shapeLabel='clabel', shapeClosed=False, start=False, tripleconstraints_list=[])}

In [14]:
dict_of_shape_objs.keys()

dict_keys(['b', 'c'])

In [50]:
# After first row, for rows that lack shapeIDs, get most-recently-inserted key from dict_of_shape_dicts
list(dict_of_shape_objs.keys())[-1]

'c'

#### 4.

In [51]:
# Problem: append multiple triple constraint dicts to tripleconstraints_list
tc_dict = dict()
tc_dict["propertyID"] = "dc:type"
tc_dict["valueConstraint"] = "foaf:Person"
dict_of_shape_objs["b"].tripleconstraints_list.append(tc_dict)
dict_of_shape_objs

{'b': CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[{'propertyID': 'dc:type', 'valueConstraint': 'foaf:Person'}]),
 'c': CSVShape(shapeID='c', shapeLabel='clabel', shapeClosed=False, start=False, tripleconstraints_list=[])}

In [52]:
# Problem: append multiple triple constraint dicts to tripleconstraints_list
tc_dict = dict()
tc_dict["propertyID"] = "dc:creator"
tc_dict["valueConstraint"] = "http://example.org/person1"
tc_obj = CSVTripleConstraint(**tc_dict)
tc_obj

CSVTripleConstraint(propertyID='dc:creator', valueConstraint='http://example.org/person1', valueShape=None, extras=None)

In [53]:
CSVTripleConstraint(**tc_dict)
dict_of_shape_objs

{'b': CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[{'propertyID': 'dc:type', 'valueConstraint': 'foaf:Person'}]),
 'c': CSVShape(shapeID='c', shapeLabel='clabel', shapeClosed=False, start=False, tripleconstraints_list=[])}

In [54]:
# This is to pretty-print the entire CSVShape
vars(CSVShape(shapeID='b', shapeLabel='label', shapeClosed=False, start=True, tripleconstraints_list=[
{'propertyID': 'dc:type', 'valueConstraint': 'foaf:Person'}, 
{'propertyID': 'dc:creator', 'valueConstraint': 'http://example.org/person1'}]))

{'shapeID': 'b',
 'shapeLabel': 'label',
 'shapeClosed': False,
 'start': True,
 'tripleconstraints_list': [{'propertyID': 'dc:type',
   'valueConstraint': 'foaf:Person'},
  {'propertyID': 'dc:creator',
   'valueConstraint': 'http://example.org/person1'}]}